In [3]:
import os
os.chdir("..")

In [ ]:
# FULL TEST
from refit_final_exp import *
full_test()

In [4]:

import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator
from GridPatternBuilder import build_habit_groups

import plotly.graph_objects as go
from plotly.subplots import make_subplots
root_folder = os.path.join(DATA_ROOT, REFIT_ROOT)


test_projects = [
    # "House1",


    "House3",
    "House4",
    "House8",
    "House9",
    "House15",


    # "House18",
    # "House20",
]
time_step = 1
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # }
        })
capacity = {
    "TV":1,
    "WashingMachine":1,
    "PC": 1,
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 4e-4
test_dates = generate_test_date(root_folder, test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
grid_pattern_cfg = {
    # "time_delta" : timedelta(minutes=10),
    "context_info" : ctx_info,
    "alpha": ccp_alpha,
    "test_dates": test_dates,
    "device_state_map": device_state_map,
    "time_delta": timedelta(minutes=time_step),
}


In [ ]:
habit_groups = {}
grid_data = {}
for p in test_projects:
    grid_data[p] = test_umass(
                root_folder = root_folder, 
                test_project=p, 
                ctx_info=ctx_info, 
                grid_cfg = grid_pattern_cfg,
                is_sim=BOOL_SIM, 
                is_umass=BOOL_UMASS)

In [ ]:
# print(len(grid_data["House3"]["TV"]))
# print(ctx_info.get_all_ctx_ordered())
# habit = {}
# for x in grid_data["House3"]["TV"]:
#     if int(x["coor"][0]) not in habit:
#         habit[int(x["coor"][0])] = []
#     habit[int(x["coor"][0])].append(x["distribution"][1]/ sum(x["distribution"]))

# print([sum(habit[x])/len(habit[x]) for x in habit])

In [ ]:
for p in test_projects:
    habit_groups[p] = build_habit_groups(grid_data[p], 1e-4)


In [ ]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [ ]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # if d != "TV":
        #     continue
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])

# for home, groups in habit_groups.items():
#     for d, d_groups in groups.items():
#         for i, g in enumerate(d_groups):
#             for j in range(i+1, len(d_groups)):
#                 box_1 = g["box"][0] + g["box"][1]
#                 box_2 = d_groups[j]["box"][0] + d_groups[j]["box"][1]
#                 if does_intersect(box_1, box_2):
#                     print("!!!")

In [ ]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

In [ ]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(root_folder, p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

In [ ]:

# TEST_RATIO = 0.3
# conflict_time = [x['cur_time'] for x in conflicts]
# end_time = max(conflict_time)
# start_time = min(conflict_time)
# print(start_time)
# print(end_time)
# total_time_range = end_time - start_time
# test_start = end_time - total_time_range * TEST_RATIO
# print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 20
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][users][s])
    if test_state_cnt[d][users][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

In [ ]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0,0] for d in all_devices}
max_p = 0
all_state_cnt = 0
for d in all_devices:
    gt_probs = []
    o_static = [[0,0], [0,0]]
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        it = np.nditer(test_state_cnt[d][u_pair_set], flags=['multi_index'])
        for count in it:
            all_state_cnt += count
            if count < MIN_TEST_OBS:
                continue
            state = it.multi_index
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (u_pair_set in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][u_pair_set]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            gt_probs.append(gt_prob)
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][2] += pred_prob - gt_prob
                exp_result[d][3] += 1
        if len(gt_probs) == 0:
            continue
        avg_gt = sum(gt_probs) / len(gt_probs)
        errors = [abs(x-avg_gt) for x in gt_probs]
        for i, e in enumerate(errors):
            if gt_probs[i] > 0.:
                o_static[0][0] += e
                o_static[0][1] += 1
            else:
                o_static[1][0] += e
                o_static[1][1] += 1
        gt_probs = []
    print("Baseline for {}".format(d))
    print("Baseline, optimal single prediction for conf is {}".format(o_static[0][0] / o_static[0][1]))
    print("Baseline, optimal single prediction for non-conf is {}".format(o_static[1][0] / o_static[1][1]))
    print("The overall baseline is {}".format((o_static[0][0] + o_static[1][0])/(o_static[0][1] + o_static[1][1])))

for d in exp_result:
    r = exp_result[d]
    o_acc = [[0., 0.], [0., 0.]]
    o_zero = 0.
    print("Device!! {}".format(d))
    print("The no. conf {}, no. non-conf {}".format(r[1], r[3]))
    print("The overall accuracy for conf is {}".format(r[0] / r[1]))
    print("The overall accuracy for non-conf is {}".format(r[2] / r[3]))
    print("The overall acc is {}".format((r[0] + r[2])/(r[1] + r[3])))

    print("The overall zero for conf is {}".format(r[4] / r[1]))
    print("The overall zero for all is {}".format(r[4] / (r[1] + r[3])))
    r.append((r[0] + r[2]) / (r[1] + r[3]))
    if r[1] > 0:
        r[0] = r[0] / r[1]
        r[4] = r[4] / r[1]
    if r[3] > 0:
        r[2] = r[2] / r[3]
    
print(max_p)
print(exp_result)

Run grid search for the optimal ccp_alpha

In [5]:
from refit_alpha_test import *

ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "weatherDesc#CAT": {},
            # WEEKDAY_CTX: {
            #     "range": (0, 6),
            #     "interval": 1,
            # },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })


def alpha_nonLinear_generator(alpha_min, alpha_max, alpha_step):
    auc_values = []
    alpha = alpha_min
    while alpha <= alpha_max:
        yield alpha
        if alpha_step <= alpha / 10:
            alpha_step *= 10
        alpha += alpha_step

print("Run test accuracy for context " + str(ctx_info.get_all_ctx_ordered()))
alpha_generator = alpha_nonLinear_generator(1e-6, 0.03, 1e-6)
acc_values = test_acc_alpha(ctx_info, 
                            gt_ctx_info, 
                            root_folder, 
                            alpha_generator, 
                            device_mapping=device_state_map,
                            time_step = 1,
                            test_projects = test_projects,
                            capacity = capacity,
                            acc_device = "TV")


Run test accuracy for context ['min_of_day#NUM']
Finish alpha 1e-06, result (0.06656198090685798, 0.000367690007725903, 0.0005991385773032879)
Finish alpha 2e-06, result (0.06656198090685798, 0.0003677055956658116, 0.0005991541107399102)
Finish alpha 3e-06, result (0.06656198090685798, 0.00036650208212181173, 0.0005979548052852248)
Finish alpha 4e-06, result (0.06656198090685798, 0.0003665636322134096, 0.0005980161401667124)
Finish alpha 4.9999999999999996e-06, result (0.06656198090685798, 0.00036647137500537186, 0.000597924205536325)
Finish alpha 5.999999999999999e-06, result (0.06609002944309508, 0.0004276931216557652, 0.0006572817101922663)
Finish alpha 6.999999999999999e-06, result (0.06609002944309508, 0.00042773714147969214, 0.000657325576100725)
Finish alpha 8e-06, result (0.06681303785481983, 0.00043027401309977893, 0.000662381578979919)
Finish alpha 9e-06, result (0.06681303785481983, 0.00043027401309977893, 0.000662381578979919)
Finish alpha 1e-05, result (0.06681303785481983